In [1]:
using ProgressMeter
using JLD
using Random
include("../alpcahus.jl")
include("../helperFunctions.jl")
include("../cluster.jl")

clusterAssignment (generic function with 1 method)

In [2]:
D = 25 # ambient space dimension
d = 3 # subspace dimension
goodPoints = 6 # good points per subspace
ν1 = 0.1 # noise variance good points
K = 2 # subspace count
varianceRatio = range( 1,300,5) # variance ratio for heatmaps
pointRatio = range(1,50,5) # point ratio between good bad data for heatmaps
trials = 50; # smoother results
qValue = 24;

In [3]:
seedVector1 = rand(Xoshiro(1234), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector2 = rand(Xoshiro(4321), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector3 = rand(Xoshiro(2143), 1:100000, trials*length(varianceRatio)*length(pointRatio));
seedVector4 = rand(Xoshiro(3412), 1:100000, trials*length(varianceRatio)*length(pointRatio));

In [4]:
trial1Tensor = zeros(length(varianceRatio), length(pointRatio),trials)
trial2Tensor = zeros(length(varianceRatio), length(pointRatio),trials)
trial3Tensor = zeros(length(varianceRatio), length(pointRatio),trials)

count = 1
@showprogress 1 "Running..." for i = 1:length(varianceRatio) # varianceRatio
     for j = 1:length(pointRatio) # prepend Threads.@threads for multi-threading
        for k = 1:trials
            # other parameters
            ν2 = varianceRatio[i]*ν1
            badPoints = round(Int, pointRatio[j]*goodPoints)
            # subspace generation
            U1 = generateSubspace(D, d; seed=seedVector1[count]) 
            U2 = generateSubspace(D, d; seed=seedVector2[count])
            # data generation
            X1 = generateData(U1, [ν1,ν2], [goodPoints,badPoints]; seed=seedVector3[count])
            X2 = generateData(U2, [ν1,ν2], [goodPoints,badPoints]; seed=seedVector4[count])
            # complete data matrix
            X = hcat(X1,X2);
            # cluster assignments
            trueLabels = vec([1*ones(size(X1)[2],1); 2*ones(size(X2)[2],1)]);
            #  trial
            estimatedCluster = ALPCAHUS(X,K, d; B=128, q=qValue, T=3, alpcahIter=10, subspaceMethod=:alpcah, finalStep=true, varfloor=1e-1, varianceMethod=:groupless)
            trial1Tensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            #  trail
            estimatedCluster = ALPCAHUS(X,K, d; B=128, q=qValue, T=3, alpcahIter=10, subspaceMethod=:alpcah, finalStep=true, varfloor=1e-5, varianceMethod=:groupless) 
            trial2Tensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            # noisy oracle trial       
            estimatedCluster = ALPCAHUS(X,K, d; B=128, q=qValue, T=3, alpcahIter=10, subspaceMethod=:alpcah, finalStep=true, varfloor=1e-9, varianceMethod=:groupless)
            trial3Tensor[i,j,k] = clusterError(trueLabels,estimatedCluster)
            # increase count
            count = count + 1
        end
    end
end

save("trialData.jld", "trial1", trial1Tensor, "trial2", trial2Tensor, "trial3", trial3Tensor);

Running... 100%|█████████████████████████████████████████| Time: 0:42:54


In [7]:
trialData = load("trialData.jld")
trial1Tensor = trialData["trial1"]
trial2Tensor = trialData["trial2"]
trial3Tensor = trialData["trial3"];

In [5]:
generateHeatmap(pointRatio, varianceRatio, trial1Tensor; plotType=:median, methodType="ALPCAHUS (varfloor=1e-1)", ν1=ν1, goodPoints=goodPoints)
savefig("trial1_median.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial1_median.pdf"

In [6]:
generateHeatmap(pointRatio, varianceRatio, trial2Tensor; plotType=:median, methodType="ALPCAHUS (varfloor=1e-5)", ν1=ν1, goodPoints=goodPoints)
savefig("trial2_median.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial2_median.pdf"

In [7]:
generateHeatmap(pointRatio, varianceRatio, trial3Tensor; plotType=:median, methodType="ALPCAHUS (varfloor=1e-9)", ν1=ν1, goodPoints=goodPoints)
savefig("trial3_median.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial3_median.pdf"

In [8]:
generateHeatmap(pointRatio, varianceRatio, trial1Tensor; plotType=:variance, methodType="ALPCAHUS (varfloor=1e-1)", ν1=ν1, goodPoints=goodPoints)
savefig("trial1_variance.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial1_variance.pdf"

In [9]:
generateHeatmap(pointRatio, varianceRatio, trial2Tensor; plotType=:variance, methodType="ALPCAHUS (varfloor=1e-5)", ν1=ν1, goodPoints=goodPoints)
savefig("trial2_variance.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial2_variance.pdf"

In [10]:
generateHeatmap(pointRatio, varianceRatio, trial3Tensor; plotType=:variance, methodType="ALPCAHUS (varfloor=1e-9)", ν1=ν1, goodPoints=goodPoints)
savefig("trial3_variance.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial3_variance.pdf"

In [11]:
generateHeatmap(pointRatio, varianceRatio, trial1Tensor; plotType=:mean, methodType="ALPCAHUS (varfloor=1e-1)", ν1=ν1, goodPoints=goodPoints)
savefig("trial1_mean.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial1_mean.pdf"

In [12]:
generateHeatmap(pointRatio, varianceRatio, trial2Tensor; plotType=:mean, methodType="ALPCAHUS (varfloor=1e-5)", ν1=ν1, goodPoints=goodPoints)
savefig("trial2_mean.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial2_mean.pdf"

In [13]:
generateHeatmap(pointRatio, varianceRatio, trial3Tensor; plotType=:mean, methodType="ALPCAHUS (varfloor=1e-9)", ν1=ν1, goodPoints=goodPoints)
savefig("trial3_mean.pdf")

"/home/javier/Desktop/alpcahus/variance_floor/trial3_mean.pdf"